In [1]:
from autotst.reaction import *
from autotst.molecule import *
from autotst.geometry import *

thermo.py:836 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:836 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:836 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:853 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
database.py:167 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-database/input/kinetics/families
statmech.py:526 loadGroups INFO Loading frequencies group database from /Users/nathan/Code/RMG-database/input/statmech/groups...
database.py:127 l

In [2]:
test_reaction = AutoTST_Reaction("OO+[CH2]CCCC_[O]O+CCCCC", "H_Abstraction")
test_reaction.ts.view_ts()

reaction.py:408 bm_pre_edit INFO Changing lower limit 3.15 to 2.812396
reaction.py:408 bm_pre_edit INFO Changing lower limit 2.519563 to 2.456764
reaction.py:408 bm_pre_edit INFO Changing lower limit 3.65 to 3.22113913735
reaction.py:467 rd_embed INFO RDKit failed to embed on attempt 1 of 10000


In [3]:
from ase.calculators.gaussian import *

In [4]:
def opt_rxn_shell(self):
    
    indicies = []

    for i, atom in enumerate(self.ts.rmg_ts.atoms):
        #print atom, i
        if not atom.label:
            indicies.append(i)
            
    c = list(itertools.combinations(indicies, 2))
    
    combos = []
    for combo in c:
        i,j = combo
        combos.append("{0} {1} F".format(i,j))
    

    self.ts.rmg_ts.updateMultiplicity()
    calc = Gaussian(mem="5GB", 
                    nprocshared="20", 
                    label="test", 
                    scratch="/gss_gpfs_scratch/harms.n/drug_conformer", 
                    method="m062x", 
                    basis="6-311+g(2df,2p)",
                    extra="opt=(ts,calcfc,noeigentest) freq",
                    multiplicity = test_reaction.ts.rmg_ts.multiplicity,
                    addsec = combos)
    del calc.parameters['force']
    return calc


def opt_rxn_center(self):
    
    indicies = []

    for i, atom in enumerate(self.ts.rmg_ts.atoms):
        #print atom, i
        if atom.label:
            indicies.append(i)
            
    c = list(itertools.combinations(indicies, 2))
    
    combos = []
    for combo in c:
        i,j = combo
        combos.append("{0} {1} F".format(i,j))
    

    self.ts.rmg_ts.updateMultiplicity()
    calc = Gaussian(mem="5GB", 
                    nprocshared="20", 
                    label="test", 
                    scratch="/gss_gpfs_scratch/harms.n/drug_conformer", 
                    method="m062x", 
                    basis="6-311+g(2df,2p)",
                    extra="opt=(ts,calcfc,noeigentest) freq",
                    multiplicity = test_reaction.ts.rmg_ts.multiplicity,
                    addsec = combos)
    del calc.parameters['force']
    return calc

In [5]:
calc = opt_rxn_shell(test_reaction)
calc.parameters

{'basis': '6-311+g(2df,2p)',
 'charge': 0,
 'mem': '5GB',
 'method': 'm062x',
 'multiplicity': 2,
 'nprocshared': '20'}

In [6]:
bond_list = []
for bond in test_reaction.ts.rdkit_ts.GetBonds():
    bond_list.append(str(bond.GetBeginAtomIdx()+1)+" " +  str(bond.GetEndAtomIdx() + 1))
bond_list

['1 2',
 '1 3',
 '2 4',
 '5 6',
 '5 7',
 '5 10',
 '5 11',
 '6 13',
 '6 8',
 '6 12',
 '7 14',
 '7 9',
 '7 15',
 '8 16',
 '8 17',
 '8 18',
 '9 20',
 '9 19']

In [7]:
calc.calculate(test_reaction.ts.ase_ts)

RuntimeError: Gaussian in . returned an error: 127

# Todo:

- [ ] Write a TS geometry optimization file
- [ ] Write a TS geometry optimization file with certian distances frozen
- [ ] Write a r / p geometry optimization file